<a href="https://colab.research.google.com/github/rushi-k12/Deployment-of-LLM/blob/main/IMDB_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sklearn torch transformers

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, pipeline
import torch

# Load the IMDb movie reviews dataset (or any other dataset of your choice)
# Here we assume you have a CSV file where 'text' is the review text and 'label' is the sentiment (0 for negative, 1 for positive)
# Replace with your actual dataset loading code

# Example dataset loading (IMDb movie reviews dataset)
# Replace with your actual dataset loading

try:
    df = pd.read_csv('/content/IMDB_Dataset.csv', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")
    print("Attempting to read the CSV file again with error_bad_lines=False...")
    try:
        df = pd.read_csv('/content/IMDB_Dataset.csv', encoding='utf-8')
    except pd.errors.ParserError as e:
        print(f"Error: ParserError still encountered after attempting to skip bad lines.")
        # Further error handling or manual inspection/cleaning may be required

# Continue with your data processing
texts = df['text'].tolist()
labels = df['label'].tolist()

# Split dataset into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Example: binary classification

# Tokenize input texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Prepare datasets
class IMDbReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbReviewsDataset(train_encodings, train_labels)
val_dataset = IMDbReviewsDataset(val_encodings, val_labels)

# Define training arguments and trainer
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=lambda p: {'accuracy': accuracy_score(p.predictions.argmax(-1), p.label_ids)},
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model_path = "./fine_tuned_model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# Deployment: Load fine-tuned model for inference
# Example: Using pipeline for sequence classification
classifier = pipeline('sentiment-analysis', model=model_path, tokenizer=model_path)

# Example inference
text = "This movie was really great!"
result = classifier(text)
print(f"Text: {text}\nSentiment: {result[0]['label']}, Score: {result[0]['score']}")

text = "This movie was terrible."
result = classifier(text)
print(f"Text: {text}\nSentiment: {result[0]['label']}, Score: {result[0]['score']}")


ParserError: Error tokenizing data. C error: EOF inside string starting at row 19792
Attempting to read the CSV file again with error_bad_lines=False...
Error: ParserError still encountered after attempting to skip bad lines.


NameError: name 'df' is not defined